In [1]:
import pandas as pd
import numpy as np

from IPython.display import display
import datetime
import plotly.express as px
from plotly.colors import sequential
from plotly.colors import diverging

from plotly.subplots import make_subplots

import math

In [2]:
matches_table = pd.read_csv('data/matches.csv')
display(matches_table)

,Date,Format,Match Number,Stage,Player 1,Player 2,Score 1,Score 2
0,2023-11-18,[Gen 9] OU,1,Group,Evan Sooklal,Will Simpson,2,6
1,2023-11-18,[Gen 9] OU,2,Group,Paul Bartenfeld,Roman Ramirez,2,6
2,2023-11-18,[Gen 9] OU,3,Group,Roman Ramirez,Evan Sooklal,6,3
3,2023-11-18,[Gen 9] OU,4,Group,Will Simpson,Paul Bartenfeld,6,1
4,2023-11-18,[Gen 9] OU,5,Group,Paul Bartenfeld,Evan Sooklal,1,6
...,...,...,...,...,...,...,...,...
105,2024-02-18,[Gen 9] Doubles UU,16,Group,Evan Sooklal,Roman Ramirez,0,6
106,2024-02-18,[Gen 9] Doubles UU,17,Group,Roman Ramirez,Aaron Carter,0,6
107,2024-02-18,[Gen 9] Doubles UU,18,Group,Jack Massingill,Will Simpson,0,6
108,2024-02-18,[Gen 9] Doubles UU,19,Group,Will Simpson,Roman Ramirez,0,6


In [3]:
updated_time = f'<i>Updated {str(datetime.datetime.now().strftime("%A, %b %d, %Y %H:%M:%S"))} CT</i>'
link_fig = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/index.html'>ELO Graph</a>"
link_fig1 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig1.html'>H2H Stats</a>"
link_fig2 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig2.html'>H2H Advanced Stats</a>"
link_fig3 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig3.html'>H2H More Stats</a>"
link_fig4 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig4.html'>Player Data</a>"
link_fig5 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig5.html'>Matches Data</a>"
link_fig6 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig6.html'>Money-Lines</a>"

In [4]:
set_player1 = set(matches_table['Player 1'])
set_player2 = set(matches_table['Player 2'])
set_players = set_player1.union(set_player2)
num_players = len(set_players)
display(set_players)

{'Aaron Carter',
 'Evan Sooklal',
 'Jack Massingill',
 'Paul Bartenfeld',
 'Roman Ramirez',
 'Will Simpson'}

In [5]:
# elo stuff
K = 31
base = 3

def expected_score(ratingA, ratingB, base=None):
    # game variables
    # base = 10
    normal_elo_difference = 100

    return (1 / (1 + np.power(base, (ratingB - ratingA) / normal_elo_difference)))

def rating_change(score, expected_score, K=None):
    # k-factor: determines how strongly a result affects the rating change
    # usually between 10 and 40, bit with a lot of games, we want to change it often
    # K = 32
    return K * (score - expected_score)

def sort_index_by_elo(table, elo):
    for i in range(2):
        table.sort_index(axis=i, key=(lambda x: [elo[y] for y in x.values]), inplace=True)

def add_elo_to_index(table):
    table.index= [f"{x}<br>{(elo[x]):,.0f}" for x in table.index]
    table.columns = [f"{x}<br>({(elo[x]):,.0f})" for x in table.columns]

def generate_elo(K, base):
    # ELO INITIALIZATION
    starting_elo = 1200.0
    elo = dict.fromkeys(set_players, starting_elo)
    elo_time = np.zeros([len(matches_table) + 1, num_players])
    elo_time_table = pd.DataFrame(elo_time)
    elo_time_table.columns = sorted(elo)
    elo_time_table.replace(0, np.NaN, inplace=True)

    prev_elo_time_table = elo_time_table.copy(deep=True)
    d_elo_time_table = elo_time_table.copy(deep=True)
    exp_elo_time_table = elo_time_table.copy(deep=True)
    wl_time_table = elo_time_table.copy(deep=True)

    record_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]),dtype=object)
    record_table.index = list(set_players)
    record_table.columns = list(set_players)

    luck_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]))
    luck_table.index = list(set_players)
    luck_table.columns = list(set_players)

    for c in record_table.columns:
        for r in record_table.index:
            record_table.at[r, c] = np.array([0, 0])

    plus_minus_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]),dtype=object)
    plus_minus_table.index = list(set_players)
    plus_minus_table.columns = list(set_players)
    
    for c in plus_minus_table.columns:
        for r in plus_minus_table.index:
            plus_minus_table.at[r, c] = np.array([0, 0])

    elo_time_table.iloc[0,:] = starting_elo
    prev_elo_time_table.iloc[0,:] = starting_elo

    for (i, row) in matches_table.iterrows():
        elo_p1 = elo[row['Player 1']]
        elo_p2 = elo[row['Player 2']]

        prev_elo_time_table.loc[i + 1, row['Player 1']] = elo_p1
        prev_elo_time_table.loc[i + 1, row['Player 2']] = elo_p2
        
        win_prob_p1 = expected_score(elo_p1, elo_p2, base=base)
        win_prob_p2 = expected_score(elo_p2, elo_p1, base=base)
        
        exp_elo_time_table.loc[i + 1, row['Player 1']] = win_prob_p1
        exp_elo_time_table.loc[i + 1, row['Player 2']] = win_prob_p2

        rating_change_p1 = rating_change(row['Score 1'] > row['Score 2'], win_prob_p1, K=K)
        rating_change_p2 = rating_change(row['Score 2'] > row['Score 1'], win_prob_p2, K=K)

        if rating_change_p1 > 0:
            luck_table.loc[row['Player 1'], row['Player 2']] += 1 - win_prob_p1
            luck_table.loc[row['Player 2'], row['Player 1']] += 0 - win_prob_p2
        elif rating_change_p2 > 0:
            luck_table.loc[row['Player 1'], row['Player 2']] += 0 - win_prob_p1
            luck_table.loc[row['Player 2'], row['Player 1']] += 1 - win_prob_p2

        plus_minus_table.loc[row['Player 1'], row['Player 2']][0] += row['Score 1']
        plus_minus_table.loc[row['Player 1'], row['Player 2']][1] += row['Score 2']
        plus_minus_table.loc[row['Player 2'], row['Player 1']][0] += row['Score 2']
        plus_minus_table.loc[row['Player 2'], row['Player 1']][1] += row['Score 1']

        if row['Score 1'] > row['Score 2']:
            record_table.loc[row['Player 1'], row['Player 2']][0] += 1
            record_table.loc[row['Player 2'], row['Player 1']][1] += 1
            wl_time_table.loc[i + 1, row['Player 1']] = row['Score 1'] - row['Score 2']
            wl_time_table.loc[i + 1, row['Player 2']] = row['Score 2'] - row['Score 1']
        elif row['Score 2'] > row['Score 1']:
            record_table.loc[row['Player 2'], row['Player 1']][0] += 1
            record_table.loc[row['Player 1'], row['Player 2']][1] += 1
            wl_time_table.loc[i + 1, row['Player 1']] = row['Score 1'] - row['Score 2']
            wl_time_table.loc[i + 1, row['Player 2']] = row['Score 2'] - row['Score 1']

        d_elo_time_table.loc[i + 1, row['Player 1']] = rating_change_p1
        d_elo_time_table.loc[i + 1, row['Player 2']] = rating_change_p2

        elo[row['Player 1']] += rating_change_p1
        elo[row['Player 2']] += rating_change_p2

        elo_time_table.loc[i + 1, row['Player 1']] = elo[row['Player 1']]
        elo_time_table.loc[i + 1, row['Player 2']] = elo[row['Player 2']]

    prev_elo_time_table = prev_elo_time_table.iloc[1:,:]
    elo_time_table = elo_time_table.iloc[1:,:]
    d_elo_time_table = d_elo_time_table.iloc[1:,:]
    exp_elo_time_table = exp_elo_time_table.iloc[1:,:]
    wl_time_table = wl_time_table.iloc[1:,:]

    list_players = sorted(set_players, key=lambda x: elo[x], reverse=False)

    for table in [record_table, luck_table, plus_minus_table]:
        sort_index_by_elo(table, elo)

    return (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, wl_time_table, record_table, luck_table, plus_minus_table)

(elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, wl_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)

luck_per_game_table = luck_table.copy(deep=True)

for c in luck_per_game_table.columns:
    for r in luck_per_game_table.index:
        if r !=c :
            luck_per_game_table.loc[r, c] /= record_table.loc[r, c].sum()
        else:
            luck_per_game_table.loc[r, c] = np.NaN


display(record_table)
display(luck_per_game_table)
display(plus_minus_table)
display(wl_time_table)

prev_elo_time_table.to_excel('exports/prev_elo_time_table.xlsx')
d_elo_time_table.to_excel('exports/d_elo_time_table.xlsx')
elo_time_table.to_excel('exports/elo_time_table.xlsx')
exp_elo_time_table.to_excel('exports/exp_elo_time_table.xlsx')



C:\Users\justr\AppData\Local\Temp\ipykernel_11304\2719908281.py:131: RuntimeWarning: invalid value encountered in scalar divide
  luck_per_game_table.loc[r, c] /= record_table.loc[r, c].sum()


,Paul Bartenfeld,Evan Sooklal,Roman Ramirez,Jack Massingill,Will Simpson,Aaron Carter
Paul Bartenfeld,"[0, 0]","[1, 5]","[2, 4]","[0, 0]","[0, 6]","[1, 3]"
Evan Sooklal,"[5, 1]","[0, 0]","[9, 4]","[1, 1]","[3, 11]","[4, 9]"
Roman Ramirez,"[4, 2]","[4, 9]","[0, 0]","[1, 1]","[7, 8]","[3, 8]"
Jack Massingill,"[0, 0]","[1, 1]","[1, 1]","[0, 0]","[1, 1]","[1, 1]"
Will Simpson,"[6, 0]","[11, 3]","[8, 7]","[1, 1]","[0, 0]","[5, 9]"
Aaron Carter,"[3, 1]","[9, 4]","[8, 3]","[1, 1]","[9, 5]","[0, 0]"


,Paul Bartenfeld,Evan Sooklal,Roman Ramirez,Jack Massingill,Will Simpson,Aaron Carter
Paul Bartenfeld,NaN,-0.179256,-0.054102,NaN,-0.301516,-0.061861
Evan Sooklal,0.179256,NaN,0.145432,0.079347,-0.186905,-0.057581
Roman Ramirez,0.054102,-0.145432,NaN,0.096126,0.094057,-0.011689
Jack Massingill,NaN,-0.079347,-0.096126,NaN,0.122440,0.260322
Will Simpson,0.301516,0.186905,-0.094057,-0.122440,NaN,-0.107992
Aaron Carter,0.061861,0.057581,0.011689,-0.260322,0.107992,NaN


,Paul Bartenfeld,Evan Sooklal,Roman Ramirez,Jack Massingill,Will Simpson,Aaron Carter
Paul Bartenfeld,"[0, 0]","[24, 35]","[21, 31]","[0, 0]","[16, 36]","[15, 21]"
Evan Sooklal,"[35, 24]","[0, 0]","[54, 43]","[6, 6]","[35, 65]","[44, 60]"
Roman Ramirez,"[31, 21]","[43, 54]","[0, 0]","[6, 6]","[61, 60]","[33, 43]"
Jack Massingill,"[0, 0]","[6, 6]","[6, 6]","[0, 0]","[6, 6]","[6, 6]"
Will Simpson,"[36, 16]","[65, 35]","[60, 61]","[6, 6]","[0, 0]","[38, 53]"
Aaron Carter,"[21, 15]","[60, 44]","[43, 33]","[6, 6]","[53, 38]","[0, 0]"


,Aaron Carter,Evan Sooklal,Jack Massingill,Paul Bartenfeld,Roman Ramirez,Will Simpson
1,NaN,-4.0,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,-4.0,4.0,NaN
3,NaN,-3.0,NaN,NaN,3.0,NaN
4,NaN,NaN,NaN,-5.0,NaN,5.0
5,NaN,5.0,NaN,-5.0,NaN,NaN
...,...,...,...,...,...,...
106,NaN,-6.0,NaN,NaN,6.0,NaN
107,6.0,NaN,NaN,NaN,-6.0,NaN
108,NaN,NaN,-6.0,NaN,NaN,6.0
109,NaN,NaN,NaN,NaN,6.0,-6.0


In [6]:
# player stats

player_table = pd.DataFrame(np.zeros([len(set_players), 0]))
player_table.index = set_players
player_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), ascending=False, inplace=True)

# ELO

player_table['Current ELO'] = [elo[x] for x in player_table.index]
player_table['Max ELO'] = elo_time_table.replace(to_replace=np.NaN, value=-np.inf).max(axis=0)
player_table['Min ELO'] = elo_time_table.replace(to_replace=np.NaN, value=np.inf).min(axis=0)

date2matches_len = dict.fromkeys(set(matches_table['Date']),0)
date2matches_first = dict.fromkeys(set(matches_table['Date']),0)
for (i, row) in matches_table.iterrows():
    if date2matches_len[row['Date']] == 0:
        date2matches_first[row['Date']] = i
    date2matches_len[row['Date']] += 1

# CHAMPIONSHIPS AND RUNNER UPS

championships = dict.fromkeys(set_players, 0)
runner_ups = dict.fromkeys(set_players, 0)
for date in set(matches_table['Date']):
    row = matches_table.iloc[date2matches_first[date] + date2matches_len[date] - 1, :]
    if row['Score 1'] > row['Score 2']:
        championships[row['Player 1']] += 1
        runner_ups[row['Player 2']] += 1
    elif row['Score 2'] > row['Score 1']:
        championships[row['Player 2']] += 1
        runner_ups[row['Player 1']] += 1

player_table['Titles'] = pd.DataFrame(championships.values(), index=championships.keys())
player_table['Runner-Ups'] = pd.DataFrame(runner_ups.values(), index=runner_ups.keys())

# RECORDS

player_table['Record'] = record_table.sum(axis=1)
player_table['GP'] = [sum(r) for r in player_table['Record']]
player_table['W'] = [r[0] for r in player_table['Record']]
player_table['L'] = [r[1] for r in player_table['Record']]
player_table['WL'] = player_table['W'] - player_table['L']
player_table['GB'] = (player_table['WL'] - player_table['WL'].max()) / 2
player_table['PCT'] = player_table['W'] / player_table['GP']
player_table.drop(columns='Record', inplace=True)

# POINTS

player_table['Raw +/-'] = plus_minus_table.sum(axis=1)
player_table['PD'] = [r[0] - r[1] for r in player_table['Raw +/-']]
player_table['PF'] = [r[0] for r in player_table['Raw +/-']]
player_table['PA'] = [r[1] for r in player_table['Raw +/-']]
player_table['PD/G'] = player_table['PD'] / player_table['GP']
player_table['PF/G'] = player_table['PF'] / player_table['GP']
player_table['PA/G'] = player_table['PA'] / player_table['GP']
player_table.drop(columns='Raw +/-', inplace=True)

# LUCK

player_table['Luck'] = luck_table.sum(axis=1)
player_table['Luck/G'] = player_table['Luck'] / player_table['GP']

# N-POINT GAMES
npg_table = pd.DataFrame(np.zeros([len(set_players), 0]))
npg_table.index = player_table.index
sort_index_by_elo(npg_table, elo)

for i in range(6):
    npg_table[f'{i+1}-PT G'] = [[0, 0] for x in range(len(set_players))]
for (i, row) in matches_table.iterrows():
    npg_table.loc[row['Player 1'], f"{abs(row['Score 1'] - row['Score 2'])}-PT G"][row['Score 1'] < row['Score 2']] += 1
    npg_table.loc[row['Player 2'], f"{abs(row['Score 1'] - row['Score 2'])}-PT G"][row['Score 1'] > row['Score 2']] += 1

# STREAKS

streaks = dict.fromkeys(set_players, list())

for player in streaks.keys():
    running = [0]
    for is_win in wl_time_table[player].dropna().reset_index(drop=True).apply(np.sign):
        if running[-1] == 0:
            running.append(is_win)
        elif (is_win > 0) and (running[-1] < 0):
            running.append(1)
        elif (is_win < 0) and (running[-1] > 0):
            running.append(-1)
        else:
            running.append(running[-1] + is_win)
    streaks[player] = running[1:]

streaks_table = pd.DataFrame(np.zeros([max([len(x) for x in streaks.values()]), len(set_players)]))
streaks_table.columns = set_players
streaks_table.sort_index(axis=1, key=(lambda x: [elo[y] for y in x.values]), ascending=False, inplace=True)

for player in streaks_table.columns:
    streaks_table[player] = pd.Series(streaks[player])

for x in ['1-PT G', '6-PT G']:
    player_table[x] = npg_table[x]
    player_table[x] = [f'{w}-{l}' for (w, l) in player_table[x]]


player_table['Streak'] = 0
for player in player_table.index:
    val = ""
    if streaks[player][-1] > 0:
        val += "W"
    elif streaks[player][-1] < 0:
        val += "L"
    val += str(int(np.abs(streaks[player][-1])))
    player_table.loc[player, 'Streak'] = val

player_table['L10'] = ""
for player in player_table.index:
    l10_w = 0
    l10_l = 0
    for i in range(10):
        try:
            if streaks[player][-1 - i] > 0:
                l10_w += 1
            elif streaks[player][-1 - i] < 0:
                l10_l += 1
        except IndexError:
            pass
    player_table.loc[player, 'L10'] = f"{l10_w}-{l10_l}"

player_table.drop('WL', axis=1, inplace=True)
player_table.drop('Luck', axis=1, inplace=True)

# STYLIZING

from pandas.io.formats.style import Styler

player_table_styled = Styler(player_table, cell_ids=False).format(
    {'Current ELO': '{:.0f}',
    'Max ELO': '{:.0f}',
    'Min ELO': '{:.0f}',
    'WL': '{:+,.0f}',
    'GB': '{:,.1f}',
    'PCT': '{:,.3f}',
    'PD': '{:+.0f}',
    'PD/G': '{:+,.1f}',
    'PF/G': '{:,.1f}',
    'PA/G': '{:,.1f}',
    'Luck': '{:+,.1%}',
    'Luck/G': '{:+,.1%}'
    }
)

player_table_styled.set_caption("Player Table Statistics")

styles = [
    {
        'selector': '',
        'props': [
            ('text-align', 'center'),
            ('font-size', '11pt'),
            ('border-collapse', 'collapse'),#'separated'),
            ('border', '1px solid silver')
        ]
    },
    {
        'selector': 'caption',
        'props': [('font', 'bold 20px Verdana')]
    },
    {
        'selector': 'tr:nth-child(even)',
        'props': 'background-color: #D3D3D3'
    },
    {
        'selector': 'td:hover',
        'props': [('background-color','#ffffb3')]
    },
    {
        'selector': 'th',
        'props': [
            ('font-family', 'Verdana'),
            ('background-color', '#828282'),
            ('color', '#FFFFFF'),
            ('padding-left', '5px'),
            ('padding-right', '5px')
        ]
    },
    {
        'selector': 'td',
        'props': [
            ('font-family', 'Courier New'),
            ('font-weight', 'bold'),
            ('padding-left', '5px'),
            ('padding-right', '5px')
        ]
    }
]

player_table_styled.set_table_styles(styles, overwrite=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(player_table_styled)

player_table_styled.to_html('figures/fig4.html')

matches_table_styled = Styler(matches_table, cell_ids=False)
matches_table_styled.set_caption("Matches Table")
matches_table_styled.set_table_styles(styles, overwrite=True)
matches_table_styled.to_html('figures/fig5.html')



,Current ELO,Max ELO,Min ELO,Titles,Runner-Ups,GP,W,L,GB,PCT,PD,PF,PA,PD/G,PF/G,PA/G,Luck/G,1-PT G,6-PT G,Streak,L10
Aaron Carter,1266,1327,1182,2,2,44,30,14,0.0,0.682,+47,183,136,+1.1,4.2,3.1,+4.8%,6-4,5-5,L1,6-4
Will Simpson,1239,1307,1197,2,2,51,31,20,-2.5,0.608,+34,205,171,+0.7,4.0,3.4,+2.5%,4-1,4-5,L1,4-6
Jack Massingill,1213,1227,1181,0,0,8,4,4,-8.0,0.500,+0,24,24,+0.0,3.0,3.0,+5.2%,0-0,4-4,L1,4-4
Roman Ramirez,1197,1258,1118,1,0,47,19,28,-12.5,0.404,-10,174,184,-0.2,3.7,3.9,-0.2%,1-7,5-4,W1,4-6
Evan Sooklal,1193,1252,1157,1,2,48,22,26,-10.0,0.458,-24,174,198,-0.5,3.6,4.1,-0.5%,7-5,4-4,W1,4-6
Paul Bartenfeld,1093,1184,1093,0,0,22,4,18,-15.0,0.182,-47,76,123,-2.1,3.5,5.6,-15.7%,1-2,0-0,L8,2-8


In [7]:
import plotly.graph_objects as go
import plotly.colors as pc

fig = go.Figure()

# adding player
ranking = 1
trace_colors = pc.qualitative.Bold
for (i, (player, current_rating)) in enumerate(sorted(elo.items(), key=lambda x:x[1], reverse=True)):
    fig.add_trace(go.Scatter(
        x=elo_time_table.index,
        y=elo_time_table[player],
        name=f'#{ranking} ({current_rating:.0f}) {player}',
        mode='lines+markers',
        connectgaps=True,
        text=[
            f"<br><b>Pre-Game ELO: </b>{x[1]:.0f}<br><b>Win Probability: </b>{x[0]:.1%}<br><br><b>Change in ELO:</b> {x[2]:+.0f}<br>" for x in zip(
                exp_elo_time_table[player],
                prev_elo_time_table[player],
                d_elo_time_table[player]
            )
        ],
        line=dict(
            shape='hv',
            color=trace_colors[i % len(trace_colors)]
        )
    ))
    ranking += 1

# adding highlighting by tournament
tournaments = list(sorted(set(matches_table['Date'])))
vrect_colors = ['green', 'red', 'yellow', 'blue', 'orange']
for (i, tourney) in enumerate(tournaments):
    fig.add_vrect(
        annotation_text=tourney,
        annotation_position="top left",
        x0=matches_table['Date'][matches_table['Date'] == tourney].index[0] + 0.5,
        x1=matches_table['Date'][matches_table['Date'] == tourney].index[-1] + 1.5,
        fillcolor=vrect_colors[i % len(vrect_colors)],
        opacity=0.1,
        line_width=0
    )

fig.update_layout(
    title=f'<b>Pokémon Showdown ELO Rating System by Roman Ramirez</b><br>{updated_time}<br>{link_fig1}, {link_fig2}, {link_fig3}, {link_fig4}, {link_fig5}, {link_fig6}',
    xaxis_title='<b>Game Number</b>',
    yaxis_title='<b>ELO Rating</b>'
)

customdata = np.stack((
        list(matches_table['Player 1']),
        list(matches_table['Player 2']),
        list(matches_table['Score 1']),
        list(matches_table['Score 2']),
        matches_table['Date'],
        list(matches_table['Format']),
        list(matches_table['Match Number'])
    ), axis=-1)
hovertemplate = (
    '<i>%{customdata[4]|%A, %B %d, %Y}, Game %{x}</i><br>' +
    '<b>%{fullData.name}</b><br><br>' + 
    '<b>Format:</b> %{customdata[5]}<br>' + 
    '<b>Tournament Match:</b> %{customdata[6]}<br><br>' + 
    '<b>%{customdata[0]} vs. %{customdata[1]}</b><br>' +
    '<b>Final Score:</b> %{customdata[2]}-%{customdata[3]}<br>' + 
    '%{text}' + 
    '<b>Post-Game ELO:</b> %{y:,.0f}<br>' +
    '<extra></extra>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate,
    opacity=0.8,
    legendgrouptitle_text='<b>#<i>Rank</i> (<i>Current ELO</i>) <i>Player</i></b>'
)

fig.show()
fig.write_html("index.html")

,Paul Bartenfeld,Evan Sooklal,Roman Ramirez,Jack Massingill,Will Simpson,Aaron Carter
Paul Bartenfeld,"[0, 0]","[1, 5]","[2, 4]","[0, 0]","[0, 6]","[1, 3]"
Evan Sooklal,"[5, 1]","[0, 0]","[9, 4]","[1, 1]","[3, 11]","[4, 9]"
Roman Ramirez,"[4, 2]","[4, 9]","[0, 0]","[1, 1]","[7, 8]","[3, 8]"
Jack Massingill,"[0, 0]","[1, 1]","[1, 1]","[0, 0]","[1, 1]","[1, 1]"
Will Simpson,"[6, 0]","[11, 3]","[8, 7]","[1, 1]","[0, 0]","[5, 9]"
Aaron Carter,"[3, 1]","[9, 4]","[8, 3]","[1, 1]","[9, 5]","[0, 0]"


In [15]:
# historical table, with record_table
win_pct_table = record_table.copy(deep=True)
text_record_table = record_table.copy(deep=True)

for c in win_pct_table.columns:
    for r in win_pct_table.index:
        if c == r:
            win_pct_table.loc[r, c] = np.NaN
            text_record_table.loc[r, c] = ""
        else:
            w, l = win_pct_table.loc[r, c]
            w_pct = (w) / (w + l) if (w + l) != 0 else 0
            win_pct_table.loc[r, c] = w_pct
            text_record_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"


for c in record_table.columns:
    for r in record_table.index:
        if sum(record_table.loc[r, c]) == 0:
            text_record_table.loc[r, c] = ""
            win_pct_table.loc[r, c] = np.NaN

win_prob_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]), dtype=object)
win_prob_table.columns = set_players
win_prob_table.index = set_players
sort_index_by_elo(win_prob_table, elo)

for c in win_prob_table.columns:
    for r in win_prob_table.index:
            if r != c:
                win_prob_table.loc[r, c] = expected_score(elo[r], elo[c], base=base)
            else:
                win_prob_table.loc[r, c] = np.NaN


In [16]:
fig1 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f"<b>{x}</b>" for x in ["Historical Win Percentage", "Current Matchup Predictor"]],
    x_title=f"{link_fig}, {link_fig2}, {link_fig3}, {link_fig4}, {link_fig5}, {link_fig6}"
)

go_win_pct = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=win_pct_table,
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_record_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

go_win_prob = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=win_prob_table,
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=win_prob_table.applymap(lambda x: "" if np.isnan(x) else f"{x:.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig1.add_trace(go_win_pct, row=1, col=1)
fig1.add_trace(go_win_prob, row=1, col=2)

fig1.layout.yaxis1.title="<b>Self</b>"
fig1.layout.yaxis2.title="<b>Self</b>"
fig1.layout.xaxis1.title="<b>Other</b>"
fig1.layout.xaxis2.title="<b>Other</b>"

fig1.update_traces(hoverinfo='skip')

fig1.update_layout(
    title=f"<b>Head-to-Head Statistics</b>, {updated_time}"
)

fig1.show()
fig1.write_html('figures/fig1.html')

In [ ]:
fig3 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f"<b>{x}</b>" for x in ["Head-to-Head Plus Minus", "N-Game Historical Record"]],
    x_title=f"<br>{link_fig}, {link_fig1}, {link_fig2}, {link_fig4}, {link_fig5}, {link_fig6}"
)

text_npg_table = npg_table.copy(deep=True)

for c in npg_table.columns:
    for r in npg_table.index:
        if c == r:
            text_npg_table.loc[r, c] = ""
        else:
            w, l = npg_table.loc[r, c]
            if (w + l) == 0:
                text_npg_table.loc[r, c] = ""
            else:
                w_pct = (w) / (w + l) if (w + l) != 0 else 0
                text_npg_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"

def calc_win_pct(wl):
    if sum(wl) == 0:
        return np.NaN
    else:
        return (wl[0]) / (sum(wl))

go_ng = go.Heatmap(
    x=[x.replace(" ", "<br>")+"ames" for x in npg_table.columns],
    y=[x.replace(" ", "<br>") for x in npg_table.index],
    z=npg_table.applymap(calc_win_pct),
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_npg_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

text_plus_minus_table = plus_minus_table.copy(deep=True)

for c in plus_minus_table.columns:
    for r in plus_minus_table.index:
        if c == r:
            text_plus_minus_table.loc[r, c] = ""
        else:
            w, l = plus_minus_table.loc[r, c]
            if (w + l) == 0:
                text_plus_minus_table.loc[r, c] = ""
            else:
                w_pct = (w) / (w + l) if (w + l) != 0 else 0
                text_plus_minus_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"

go_pm = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in plus_minus_table.columns],
    y=[x.replace(" ", "<br>") for x in plus_minus_table.index],
    z=plus_minus_table.applymap(calc_win_pct),
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_plus_minus_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig3.add_trace(go_pm, row=1, col=1)
fig3.add_trace(go_ng, row=1, col=2)

fig3.layout.yaxis1.title="<b>Self</b>"
fig3.layout.yaxis2.title="<b>Self</b>"
fig3.layout.xaxis1.title="<b>Other</b>"
fig3.layout.xaxis2.title="<b>Other</b>"

fig3.update_traces(
    hoverinfo='skip'
)

fig3.update_layout(
    title=f"<b>Head-to-Head Statistics</b>, {updated_time}"
)

fig3.show()
fig3.write_html('figures/fig3.html')

In [ ]:
# difference in win percentage and matchup predictor
# positive number means overrated, negative number means underrated
# understanding: Player A is overrated/underrated against Player B

key_upset_table = -win_pct_table + win_prob_table
        
for c in record_table.columns:
    for r in record_table.index:
        if sum(record_table.loc[r, c]) == 0:
            key_upset_table.loc[r, c] = np.NaN


fig2 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        "<b>Over/Underrated</b><br>How Much Better is the Win Probability Against Record", 
        "<b>Average Luck Per Game</b><br>Average Difference in Actual and Expected Performance"
    ],
    x_title=f"{link_fig}, {link_fig1}, {link_fig3}, {link_fig4}, {link_fig5}, {link_fig6}"
)

go_ou = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=key_upset_table,
    zmax=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None), abs(min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)))),
    zmid=0,
    zmin=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None), abs(min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)))) * -1,
    text=key_upset_table.applymap(lambda x: "" if np.isnan(x) else f"{x:+.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

go_luck_pg = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=luck_per_game_table,
    zmax=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None), abs(min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)))),
    zmid=0,
    zmin=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None), abs(min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)))) * -1,
    text=luck_per_game_table.applymap(lambda x: "" if np.isnan(x) else f"{x:+.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig2.add_trace(go_ou, row=1, col=1)
fig2.add_trace(go_luck_pg, row=1, col=2)

fig2.layout.yaxis1.title="<b>Self</b>"
fig2.layout.yaxis2.title="<b>Self</b>"
fig2.layout.xaxis1.title="<b>Other</b>"
fig2.layout.xaxis2.title="<b>Other</b>"

fig2.update_traces(hoverinfo='skip')

fig2.update_layout(
    title=f"<b>Head-to-Head Advanced Statistics</b>, {updated_time}"
)

fig2.show()
fig2.write_html('figures/fig2.html')

In [ ]:
# betting/gambling table

# to-do
# odds to win championship
# record for/against the spread
# player vs. player spread
# over/under total

odds_table = win_prob_table / win_prob_table.T
ml_table = odds_table.copy(deep=True)

for c in ml_table.columns:
    for r in ml_table.index:
        ml_table.loc[r, c] = np.NaN
        win_prob = win_prob_table.loc[r, c]
        if r == c:
            pass
        elif win_prob > 0.5:
            ml_table.loc[r, c] = -1 * win_prob / (1 - win_prob) * 100
        elif win_prob < 0.5:
            ml_table.loc[r, c] = (1 - win_prob) / win_prob * 100

ml_table = ml_table.applymap(lambda x: '{:+.0f}'.format((x * 2).round(-1) / 2) if not np.isnan(x) else np.NaN)

display(odds_table)
display(ml_table)

,Paul Bartenfeld,Evan Sooklal,Roman Ramirez,Jack Massingill,Will Simpson,Aaron Carter
Paul Bartenfeld,NaN,0.334411,0.317877,0.267553,0.200743,0.149894
Evan Sooklal,2.990329,NaN,0.950556,0.80007,0.600286,0.448231
Roman Ramirez,3.145873,1.052016,NaN,0.841686,0.63151,0.471546
Jack Massingill,3.737584,1.249891,1.188091,NaN,0.750292,0.56024
Will Simpson,4.981506,1.665872,1.583505,1.332815,NaN,0.746696
Aaron Carter,6.671398,2.230992,2.120683,1.78495,1.339233,NaN


,Paul Bartenfeld,Evan Sooklal,Roman Ramirez,Jack Massingill,Will Simpson,Aaron Carter
Paul Bartenfeld,NaN,+300,+315,+375,+500,+665
Evan Sooklal,-300,NaN,+105,+125,+165,+225
Roman Ramirez,-315,-105,NaN,+120,+160,+210
Jack Massingill,-375,-125,-120,NaN,+135,+180
Will Simpson,-500,-165,-160,-135,NaN,+135
Aaron Carter,-665,-225,-210,-180,-135,NaN


In [ ]:
# best 2 out of 3 calculator
best_23_table = win_pct_table.copy(deep=True)
best_23_table = best_23_table.applymap(lambda x: 0)

for c in best_23_table.columns:
    for r in best_23_table.columns:
        if r == c:
            best_23_table.loc[r, c] = np.NaN
        else:
            win_prob = win_prob_table.loc[r, c]
            best_23_table.loc[r, c] = np.power(win_prob, 2) * (1 + (2 * (1 - win_prob)))

display(best_23_table)

,Paul Bartenfeld,Evan Sooklal,Roman Ramirez,Jack Massingill,Will Simpson,Aaron Carter
Paul Bartenfeld,NaN,0.156932,0.146472,0.114853,0.074504,0.046547
Evan Sooklal,0.843068,NaN,0.480993,0.417042,0.316563,0.228080
Roman Ramirez,0.853528,0.519007,NaN,0.435688,0.333487,0.242242
Jack Massingill,0.885147,0.582958,0.564312,NaN,0.393726,0.294207
Will Simpson,0.925496,0.683437,0.666513,0.606274,NaN,0.391998
Aaron Carter,0.953453,0.771920,0.757758,0.705793,0.608002,NaN


In [ ]:
# making playoffs odds
# assumptions
## assume ELO doesn't change after each game
## assume if you win at least 6/8 of your games, you're in for the championship
## independent on the order of matches
def prob_of_x_wins(player, num_wins=6):
    sp = set_players.copy()
    sp.remove(player)
    schedule = list(sp) + list(sp)

    total_prob = 0
    for i in range(np.power(2, 8)):
        single_prob = 1
        record = '{0:08b}'.format(i)
        if record.count('1') != (num_wins):
            continue
        for (j, x) in enumerate(record):
            if x == '1':
                single_prob *= win_prob_table.loc[player, schedule[j]]
            elif x == '0':
                single_prob *= win_prob_table.loc[schedule[j], player]
        total_prob += single_prob
    return total_prob

prob_of_x_wins_table = pd.DataFrame(np.zeros([len(set_players), 9]))
prob_of_x_wins_table.index = list(set_players)
prob_of_x_wins_table.columns
prob_of_x_wins_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), ascending=False, inplace=True)

for c in prob_of_x_wins_table.columns:
    for r in prob_of_x_wins_table.index:
        prob_of_x_wins_table.loc[r, c] = prob_of_x_wins(r, int(c))

odds_of_x_wins_table = (1 - prob_of_x_wins_table) / prob_of_x_wins_table
ml_of_x_wins_table = odds_of_x_wins_table.applymap(lambda x: '{:+,.0f}'.format(float('%.2g' % (x * 100)) if not np.isnan(x) else np.NaN))

# what is the probability of getting at least x wins?
prob_of_cum_x_wins_table = prob_of_x_wins_table.copy(deep=True)
prob_of_cum_x_wins_table.applymap(lambda x: 0)

for (i, c) in enumerate(prob_of_cum_x_wins_table.columns):
    prob_of_cum_x_wins_table.loc[:, c] = prob_of_x_wins_table.iloc[:, i:8+1].sum(axis=1)
prob_of_cum_x_wins_table.drop(columns=0, axis=1, inplace=True)

odds_of_cum_x_wins_table = (1 - prob_of_cum_x_wins_table) / prob_of_cum_x_wins_table
ml_of_cum_x_wins_table = odds_of_cum_x_wins_table.copy(deep=True)

for c in ml_of_cum_x_wins_table.columns:
    for r in ml_of_cum_x_wins_table.index:
        ml_of_cum_x_wins_table.loc[r, c] = np.NaN
        odds = odds_of_cum_x_wins_table.loc[r, c]
        if odds < 1:
            ml_of_cum_x_wins_table.loc[r, c] = -1 * (1 / odds_of_cum_x_wins_table.loc[r, c]) * 100
        elif odds > 1:
            ml_of_cum_x_wins_table.loc[r, c] = odds_of_cum_x_wins_table.loc[r, c] * 100

ml_of_cum_x_wins_table = ml_of_cum_x_wins_table.applymap(lambda x: '{:+,.0f}'.format(float('%.2g' % (x)) if not np.isnan(x) else np.NaN))

display(prob_of_x_wins_table)
display(odds_of_x_wins_table)
display(ml_of_x_wins_table)
display(prob_of_cum_x_wins_table)
display(odds_of_cum_x_wins_table)
display(ml_of_cum_x_wins_table)

,0,1,2,3,4,5,6,7,8
Aaron Carter,0.000029,0.000714,0.007261,0.039667,0.128618,0.255741,0.306629,0.203714,0.057628
Will Simpson,0.000160,0.002929,0.021839,0.086991,0.204423,0.292567,0.250404,0.117522,0.023164
Jack Massingill,0.000673,0.009197,0.051120,0.151422,0.263799,0.278917,0.175691,0.060464,0.008717
Roman Ramirez,0.001697,0.018973,0.085682,0.204715,0.286342,0.242528,0.122359,0.033784,0.003921
Evan Sooklal,0.002198,0.023147,0.098209,0.220005,0.288303,0.228799,0.108232,0.028048,0.003059
Paul Bartenfeld,0.145758,0.319256,0.303599,0.163609,0.054606,0.011548,0.001510,0.000111,0.000004


,0,1,2,3,4,5,6,7,8
Aaron Carter,34782.922430,1399.623703,136.724176,24.209974,6.774975,2.910204,2.261275,3.908843,16.352622
Will Simpson,6243.344699,340.396629,44.788604,10.495377,3.891816,2.418023,2.993551,7.509060,42.169734
Jack Massingill,1484.488393,107.734086,18.561655,5.604070,2.790769,2.585291,4.691797,15.538900,113.720122
Roman Ramirez,588.370646,51.706111,10.671028,3.884839,2.492330,3.123240,7.172688,28.599763,254.068132
Evan Sooklal,453.934076,42.201867,9.182376,3.545347,2.468577,3.370643,8.239452,34.652771,325.918080
Paul Bartenfeld,5.860703,2.132287,2.293820,5.112134,17.312975,85.593573,661.403698,8975.601246,281868.538977


,0,1,2,3,4,5,6,7,8
Aaron Carter,"+3,500,000","+140,000","+14,000","+2,400",+680,+290,+230,+390,"+1,600"
Will Simpson,"+620,000","+34,000","+4,500","+1,000",+390,+240,+300,+750,"+4,200"
Jack Massingill,"+150,000","+11,000","+1,900",+560,+280,+260,+470,"+1,600","+11,000"
Roman Ramirez,"+59,000","+5,200","+1,100",+390,+250,+310,+720,"+2,900","+25,000"
Evan Sooklal,"+45,000","+4,200",+920,+350,+250,+340,+820,"+3,500","+33,000"
Paul Bartenfeld,+590,+210,+230,+510,"+1,700","+8,600","+66,000","+900,000","+28,000,000"


,1,2,3,4,5,6,7,8
Aaron Carter,0.999971,0.999257,0.991996,0.952330,0.823712,0.567971,0.261342,0.057628
Will Simpson,0.999840,0.996911,0.975071,0.888080,0.683657,0.391090,0.140686,0.023164
Jack Massingill,0.999327,0.990130,0.939010,0.787588,0.523789,0.244872,0.069180,0.008717
Roman Ramirez,0.998303,0.979330,0.893648,0.688933,0.402591,0.160063,0.037705,0.003921
Evan Sooklal,0.997802,0.974655,0.876446,0.656441,0.368138,0.139339,0.031107,0.003059
Paul Bartenfeld,0.854242,0.534987,0.231388,0.067779,0.013173,0.001625,0.000115,0.000004


,1,2,3,4,5,6,7,8
Aaron Carter,0.000029,0.000743,0.008068,0.050057,0.214017,0.760654,2.826401,16.352622
Will Simpson,0.000160,0.003099,0.025566,0.126025,0.462723,1.556957,6.108019,42.169734
Jack Massingill,0.000674,0.009968,0.064952,0.269700,0.909165,3.083769,13.454965,113.720122
Roman Ramirez,0.001700,0.021106,0.119009,0.451520,1.483910,5.247527,25.521980,254.068132
Evan Sooklal,0.002203,0.026004,0.140972,0.523367,1.716373,6.176757,31.146918,325.918080
Paul Bartenfeld,0.170628,0.869205,3.321748,13.753855,74.913987,614.535418,8698.549710,281868.538977


,1,2,3,4,5,6,7,8
Aaron Carter,"-3,500,000","-130,000","-12,000","-2,000",-470,-130,+280,"+1,600"
Will Simpson,"-620,000","-32,000","-3,900",-790,-220,+160,+610,"+4,200"
Jack Massingill,"-150,000","-10,000","-1,500",-370,-110,+310,"+1,300","+11,000"
Roman Ramirez,"-59,000","-4,700",-840,-220,+150,+520,"+2,600","+25,000"
Evan Sooklal,"-45,000","-3,800",-710,-190,+170,+620,"+3,100","+33,000"
Paul Bartenfeld,-590,-120,+330,"+1,400","+7,500","+61,000","+870,000","+28,000,000"


In [ ]:
fig6 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        "<b>Current Matchup M/L</b><br>Odds for a Single Game", 
        "<b>M/L: Group-Stage Wins</b><br>Odds of Winning at Least <b>X</b> Games"
    ],
    x_title=f"{link_fig}, {link_fig1}, {link_fig2}, {link_fig3}, {link_fig4}, {link_fig5}"
)


go_ml = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in ml_table.columns],
    y=[x.replace(" ", "<br>") for x in ml_table.index],
    z=win_prob_table,
    zmax=max(win_prob_table.max(axis=None), win_prob_table.max(axis=None)),
    zmid=0.5,
    zmin=min(win_prob_table.min(axis=None), win_prob_table.min(axis=None)),
    text=ml_table.applymap(lambda x: "" if math.isnan(float(x)) else x),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

ml_of_cum_x_wins_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), inplace=True)
prob_of_cum_x_wins_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), inplace=True)
scale_go_ml_cum_x_wins = prob_of_cum_x_wins_table

go_ml_cum_x_wins = go.Heatmap(
    x=[(str(x)) for x in ml_of_cum_x_wins_table.columns],
    y=[x.replace(" ", "<br>") for x in ml_of_cum_x_wins_table.index],
    z=scale_go_ml_cum_x_wins,
    zmax=max(scale_go_ml_cum_x_wins.max(axis=None), scale_go_ml_cum_x_wins.max(axis=None)),
    zmid=0,
    zmin=min(scale_go_ml_cum_x_wins.min(axis=None), scale_go_ml_cum_x_wins.min(axis=None)),
    text=ml_of_cum_x_wins_table.applymap(lambda x: "" if math.isnan(float(x.replace(",", "")))  else x.replace(",", "")),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig6.add_trace(go_ml, row=1, col=1)
fig6.add_trace(go_ml_cum_x_wins, row=1, col=2)

fig6.layout.xaxis1.title="<b>Other</b>"
fig6.layout.yaxis1.title="<b>Self</b>"
fig6.layout.xaxis2.title="<b>Number of Total Wins in Group Stage</b>"
fig6.layout.yaxis2.title="<b>Self</b>"

fig6.update_traces(hoverinfo='skip', showscale=False)

fig6.update_layout(
    title=f"<b>Money-Lines</b>, {updated_time}"
)

fig6.show()
fig6.write_html('figures/fig6.html')

In [ ]:
ml_table

,Paul Bartenfeld,Evan Sooklal,Roman Ramirez,Jack Massingill,Will Simpson,Aaron Carter
Paul Bartenfeld,NaN,+300,+315,+375,+500,+665
Evan Sooklal,-300,NaN,+105,+125,+165,+225
Roman Ramirez,-315,-105,NaN,+120,+160,+210
Jack Massingill,-375,-125,-120,NaN,+135,+180
Will Simpson,-500,-165,-160,-135,NaN,+135
Aaron Carter,-665,-225,-210,-180,-135,NaN


In [ ]:
# # system optimization

# def optimization1():
#     index = [_ for _ in np.arange(1, 51, 1.0)]
#     columns = [_ for _ in np.arange(2, 11, 1.0)]

#     opt_table = pd.DataFrame(np.zeros([len(index), len(columns)]))
#     opt_table.index = index
#     opt_table.columns = columns

#     for base in opt_table.columns:
#         for K in opt_table.index:
#             (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)
#             num_upsets = 0
#             for i in elo_time_table.index:
#                 prev_elo_slice = prev_elo_time_table.loc[i,:].dropna().values
#                 elo_slice = elo_time_table.loc[i,:].dropna().values
#                 combo_elo = np.array([prev_elo_slice, elo_slice])
#                 is_upset = ((combo_elo[1, 0] - combo_elo[0, 0]) * (combo_elo[0, 1] - combo_elo[0, 0])) > 0
#                 if is_upset:
#                     num_upsets += 1
#             opt_table.loc[K,base] = num_upsets

#     fig = px.imshow(opt_table.T,labels=dict(y="Base", x="K-value", color="Number of Upsets"))#, text_auto=True, aspect="auto")
#     fig.update_layout(
#         title="<b>Total Number of Upsets against K-Value and Base</b>"
#     )
#     fig.show()
#     return opt_table
# opt1 = optimization1()
# fig.write_html("figures/opt1.html")

In [ ]:
# # system optimization

# def optimization2():
#     index = [_ for _ in np.arange(1, 51, 1.0)]
#     columns = [_ for _ in np.arange(2, 11, 1.0)]

#     opt_table = pd.DataFrame(np.zeros([len(index), len(columns)]))
#     opt_table.index = index
#     opt_table.columns = columns

#     for base in opt_table.columns:
#         for K in opt_table.index:
#             (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)
#             total_correctness = 0
#             for i in elo_time_table.index:
#                 d_elo_slice = d_elo_time_table.loc[i,:].dropna().values
#                 exp_elo_slice = exp_elo_time_table.loc[i,:].dropna().values
#                 correctness = (np.sign(d_elo_slice) * exp_elo_slice).sum()
#                 total_correctness += correctness
#             opt_table.loc[K,base] = total_correctness

#     fig = px.imshow(opt_table.T,labels=dict(y="Base", x="K-value", color="Total Correctness"))#, text_auto=False, aspect="auto")
#     fig.update_layout(
#         title="<b>Win Percentage Correctness against K-Value and Base</b>"
#     )
#     fig.show()
#     return opt_table
# opt2 = optimization2()
# fig.write_html("figures/opt2.html")

In [ ]:
# def norm_minmax(dataframe):
#     return ((dataframe - dataframe.min(axis=None)) / (dataframe.max(axis=None) - dataframe.min(axis=None)))

# norm_opt1 = norm_minmax(opt1)
# norm_opt2 = norm_minmax(opt2)

# fig = px.imshow(norm_opt2.T - norm_opt1.T,labels=dict(y="Base", x="K-value", color="Normalized Optimization"), text_auto=False, aspect="auto")
# fig.update_layout(
#     title="<b>K Value and Base against Both Optimization Functions</b>"
# )
# fig.show()
# fig.write_html("figures/opt_total.html")